In [5]:
!apt-get install -y openjdk-11-jdk
!pip install tabula-py pandas
!pip install pdfplumber


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
openjdk-11-jdk is already the newest version (11.0.27+6~us1-0ubuntu1~22.04).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 92.9 MB/s eta 0:00:00


In [6]:
import urllib.request

url = "https://download.bgmea.com.bd/Associate%20Member.pdf"
urllib.request.urlretrieve(url, "Associate_Member.pdf")
print("✅ PDF downloaded.")


✅ PDF downloaded.


In [7]:
import pdfplumber

all_text = ""

with pdfplumber.open("Associate_Member.pdf") as pdf:
    for page_num, page in enumerate(pdf.pages, start=1):
        text = page.extract_text()
        print(f"\n--- Page {page_num} ---\n")
        print(text[:1000])  # Preview first 1000 characters
        all_text += text + "\n"

# Save to raw text file for inspection
with open("associate_members_raw.txt", "w", encoding="utf-8") as f:
    f.write(all_text)

print("✅ Text extracted and saved.")



--- Page 1 ---

List of Associate Member (Buying House)
Nafiz Mutasim Billah, Managing Director Riazur Rahman, Managing Director
2Gamtex Ltd. (Reg-43) 4 Star Trade International (Reg-323)
76, DIT Road 385/1, East Rampura (6th Floor)
Malibagh, Dhaka, Rampura, Dhaka,
Mob/Tel-, 9336583 Mob/Tel-, 9340830, 8855811, 0171532301
Email-gemtex@accesstel.net Email-suman@4starfashions.com
Dr. Ezazur Rahman, Proprietor Anwar Kabir, Proprietor
7 Mark (Reg-799) A & B Apparels (Reg-58)
7-9, Kawran Bazar, BTMC Bahaban (7th Floor) House # 26, Road # 3, Block-C, Flat # 4A, Bonasree
Kawran Bazar, Dhaka, Project
Mob/Tel-, 8112523, 01711590399 Rampura, Dhaka,
Email-info@7mark.net Mob/Tel-, 7286871
Email-anbapparels@bdonline.com
Bong IL Kim, Managing Director Tultul Chowdhury, Managing Partner
A & C (BD) Ltd. (Reg-109) A & R Apparels International (Reg-1037)
House # 26 (3rd Floor), Block-G, Road # 7 6/2, Jawab Street (1st Floor)
Banani, Dhaka, Wari, Dhaka,
Mob/Tel-, 8828066, 9892549-50 Mob/Tel-, 01833311286

In [15]:
import re
import pandas as pd

# Load the full text file
with open("associate_members_raw.txt", "r", encoding="utf-8") as file:
    raw_text = file.read()


In [16]:
# This matches entries where a line contains "(Reg-###)"
raw_entries = re.findall(
    r"(?:(?:[^\n]+\n){0,1})"          # Optional contact person line
    r"([^\n]+?)\s+\(Reg-(\d+)\)\n"    # Company name + Reg No
    r"([^\n]+)\n"                     # Address line 1
    r"([^\n]+)\n"                     # Address line 2
    r"Mob/Tel-.*,?\n?"                # Phone line
    r"Email-[^\n]+\n?",               # Email line
    raw_text, re.MULTILINE)

print(f"✅ Found {len(raw_entries)} entries.")


✅ Found 745 entries.


In [17]:
data = []

for match in raw_entries:
    company, reg_no, addr1, addr2 = match

    # Now re-find the full entry text
    pattern = re.escape(company) + r"\s+\(Reg-" + re.escape(reg_no) + r"\).*?Email-([^\n]+)"
    full_match = re.search(pattern, raw_text, re.DOTALL)

    # Try to grab contact person and phone manually
    contact_match = re.search(r"([^\n]+)\n" + re.escape(company), raw_text)
    contact_person = contact_match.group(1).strip() if contact_match else ""

    phone_match = re.search(re.escape(company) + r".*?Mob/Tel-([^\n]+)", raw_text, re.DOTALL)
    phone = phone_match.group(1).strip() if phone_match else ""

    email_match = re.search(re.escape(company) + r".*?Email-([^\n]+)", raw_text, re.DOTALL)
    email = email_match.group(1).strip() if email_match else ""

    data.append({
        "Company Name": company.strip(),
        "Reg No": reg_no.strip(),
        "Contact Person": contact_person,
        "Address": f"{addr1.strip()}, {addr2.strip()}",
        "Phone": phone,
        "Email": email
    })


In [18]:
df = pd.DataFrame(data)
df.to_csv("bgmea_associate_members_full.csv", index=False)
df.head()


,Company Name,Reg No,Contact Person,Address,Phone,Email
0,2Gamtex Ltd. (Reg-43) 4 Star Trade International,323,"Nafiz Mutasim Billah, Managing Director Riazur...","76, DIT Road 385/1, East Rampura (6th Floor), ...",", 9336583 Mob/Tel-, 9340830, 8855811, 0171532301",gemtex@accesstel.net Email-suman@4starfashions...
1,7 Mark (Reg-799) A & B Apparels,58,"Dr. Ezazur Rahman, Proprietor Anwar Kabir, Pro...","7-9, Kawran Bazar, BTMC Bahaban (7th Floor) Ho...",", 8112523, 01711590399 Rampura, Dhaka,","info@7mark.net Mob/Tel-, 7286871"
2,A & C (BD) Ltd. (Reg-109) A & R Apparels Inter...,1037,"Bong IL Kim, Managing Director Tultul Chowdhur...","House # 26 (3rd Floor), Block-G, Road # 7 6/2,...",", 8828066, 9892549-50 Mob/Tel-, 01833311286, 0...",ancbd@citech-bd.com Email-ar.apparelsintl@gmai...
3,A & S Dot Com (Reg-1005) A & S Sourcing,1046,"Sumon Hamid, Proprietor Amirul Absar, Proprietor","Bengal Plaza # 4 (1st Fl), Jamgora RDR House (...",", 01819241069, 8411260.61 Mob/Tel-, 0171143908...",sumon@aandsdotcom.com Email-contact@assourcing...
4,A K Tex Trading (Reg-674) A Plus Industries.,544,"Amal Chandra Datta, Proprietor Mohiuddin Ahmed...","Section 10, Block C, Road # 18, House 01 30 Di...",", 01713009108, 01190238186 Mob/Tel-, 9353383, ...",aktex@pradeshta.net Email-aconcern@bangla.net
